In [7]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))

Overwriting strands_claude.py


In [9]:
!python strands_claude.py "{\"prompt\": \"What is the weather now?\"}"


Tool #1: weather
The current weather is sunny.


In [10]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


In [11]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=D:\Code\github\Bedrock_agent\01-AgentCore-runtime\01-hosting-agent\strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: D:\Code\github\Bedrock_agent\01-AgentCore-runtime\01-hosting-agent\Dockerfile
Generated .dockerignore: D:\Code\github\Bedrock_agent\01-AgentCore-runtime\01-hosting-agent\.dockerignore
Setting 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: D:\Code\github\Bedrock_agent\01-AgentCore-runtime\01-hosting-agent\.bedrock_agentcore.yaml


ConfigureResult(config_path=WindowsPath('D:/Code/github/Bedrock_agent/01-AgentCore-runtime/01-hosting-agent/.bedrock_agentcore.yaml'), dockerfile_path=WindowsPath('D:/Code/github/Bedrock_agent/01-AgentCore-runtime/01-hosting-agent/Dockerfile'), dockerignore_path=WindowsPath('D:/Code/github/Bedrock_agent/01-AgentCore-runtime/01-hosting-agent/.dockerignore'), runtime='None', region='ap-southeast-1', account_id='566801649110', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [12]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 566801649110 (ap-southeast-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_getting_started


✅ ECR repository available: 566801649110.dkr.ecr.ap-southeast-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: ap-southeast-1, account ID: 566801649110
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-8556fc4504
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-8556fc4504
Starting execution role creation process for agent: strands_claude_getting_started
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-8556fc4504
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-8556fc4504
✓ Role created: arn:aws:iam::566801649110:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-8556fc4504
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_getting_started
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::566801649110

In [13]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

In [14]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'ce0dc25f-b211-4ce8-a5c2-b0ed3fbadd8f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Nov 2025 04:05:15 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ce0dc25f-b211-4ce8-a5c2-b0ed3fbadd8f',
   'baggage': 'Self=1-690d6ff9-0f9246916886e27612b6a9bb,session.id=43701b0e-406c-4118-98fc-b7cdb1015b49',
   'x-amzn-bedrock-agentcore-runtime-session-id': '43701b0e-406c-4118-98fc-b7cdb1015b49',
   'x-amzn-trace-id': 'Root=1-690d6ff9-4c62a0a604ad8b130fbb0b5a;Parent=bc4e6eb616f2df4c;Sampled=1;Self=1-690d6ff9-0f9246916886e27612b6a9bb'},
  'RetryAttempts': 0},
 'runtimeSessionId': '43701b0e-406c-4118-98fc-b7cdb1015b49',
 'traceId': 'Root=1-690d6ff9-4c62a0a604ad8b130fbb0b5a;Parent=bc4e6eb616f2df4c;Sampled=1;Self=1-690d6ff9-0f9246916886e27612b6a9bb',
 'baggage': 'Self=1-690d6ff9-0f9246916886e27612b6a9bb,session.id=43701b0e-406c-4118-98fc-b7cdb1015b49',
 'contentType': 

In [15]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

The weather is sunny right now.

In [16]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The expression "2 + 2" evaluates to 4.